In [150]:
import pandas as pd
import numpy as np
import joblib
import requests

from io import BytesIO
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [151]:
def imputar_f (df,list_cat,SimpleImputer,pd):  
    columns = df.columns.values
    df_c=df[list_cat]
    df_n=df.loc[:,~df.columns.isin(list_cat)]

    imputer_n=SimpleImputer(strategy='median')
    imputer_c=SimpleImputer(strategy='most_frequent')

    imputer_n.fit(df_n)
    imputer_c.fit(df_c)

    X_n=imputer_n.transform(df_n)
    X_c=imputer_c.transform(df_c)

    df_n=pd.DataFrame(X_n,columns=df_n.columns)
    df_c=pd.DataFrame(X_c,columns=df_c.columns)

    df = pd.concat([df_n,df_c],axis=1)[columns]
    return df

In [153]:
objects = ['RandomForestModel', 'Features', 'most_frecuently', 'dummies']
for obj in objects:
  link_ = f'https://github.com/santiagogz11/HRProject/blob/main/{obj}.pkl?raw=true'
  file_ = BytesIO(requests.get(link_).content)
  globals()[obj] = joblib.load(file_)

In [154]:
DataNames = {'employee_survey_data':'employee_survey', 'general_data':'general', 'manager_survey_data':'manager_survey'}

In [155]:
for key, value in zip(DataNames.keys(), DataNames.values()):
  globals()[f'df_{value}'] = pd.read_csv(f'https://raw.githubusercontent.com/santiagogz11/HRProject/main/data/{key}.csv', engine='python', sep = None)

In [156]:
df = pd.merge(df_general, df_employee_survey, on = 'EmployeeID', how = 'left')
df = pd.merge(df, df_manager_survey, on = 'EmployeeID', how = 'left')

In [157]:
df = imputar_f(df, most_frecuently, SimpleImputer, pd)

In [158]:
noDummies = df.columns[~df.columns.isin(most_frecuently)].values

In [159]:
scaler = MinMaxScaler()
X_noDummies = scaler.fit_transform(df[noDummies])

In [160]:
X_dummies = pd.get_dummies(df[dummies])

In [164]:
X = pd.concat([pd.DataFrame(X_dummies), pd.DataFrame(X_noDummies, columns=noDummies)], axis = 1)[features]

In [165]:
predict = pd.concat([pd.DataFrame(X.index.values + 1, columns=['EmployeeID']), 
                     pd.DataFrame(RandomForestModel.predict(X), columns=['Attrition'])], axis = 1)

In [166]:
predict.query('Attrition == 1')

,EmployeeID,Attrition
1,2,1
6,7,1
13,14,1
28,29,1
30,31,1
...,...,...
4381,4382,1
4386,4387,1
4388,4389,1
4391,4392,1
